In [ ]:
# --- PARAMETER CONFIGURATION --- YOU SHOULD INSERT YOUR API KEY

!pip install -q googlemaps
!pip install -q -U google-generativeai

import googlemaps
import google.generativeai as genai

# Problem type:
# 1 = Hamiltonian Cycle (return to starting city)
# 2 = Hamiltonian Path (open path)
PROBLEM_TYPE = 2

# Gemini settings
useGemini = False
search_area = "Warsaw"
num_attractions = 8

BASE_PROMPT = (
    f"You are an expert tour guide. List exactly the {num_attractions} most iconic and interesting attractions "
    f"in {search_area} (no more, no less). Avoid generic names because I must search them on Google Maps. "
    f"Respond ONLY with: Attraction Name, Full Address, {search_area}; "
    f"(you must provide this line for each attraction). "
    f"Separate each attraction with a semicolon, without descriptions or numbering."
)

# Cost matrix settings
# metric: "distance" (meters), "duration" (seconds)
# mode: "driving", "walking", "bicycling", "transit"
cost_metric = "distance"
travel_mode = "walking"

# API keys and locations
MAPS_API_KEY = "INSERT_YOUR_KEY"
GEMINI_API_KEY = "INSERT_YOUR_KEY"

cities = [
    "Muro Originale Del Ghetto Ebraico, Warsaw", "Palazzo della Cultura e della Scienza, Warsaw", "Złota, Warsaw", "myhive Warsaw Spire, Warsaw", "Varso Tower, Warsaw"
]


In [71]:
def get_attractions_from_gemini(api_key, prompt):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel("gemini-2.0-flash")

    try:
        response = model.generate_content(prompt)
        attractions = [a.strip() for a in response.text.split(";")]
        return attractions
    except Exception as e:
        print(f"Gemini error: {e}")
        return []


if useGemini:
    cities = get_attractions_from_gemini(GEMINI_API_KEY, BASE_PROMPT)

    # Append city name to help Google Maps
    cities = [f"{c}" for c in cities]

    print("\nItinerary proposed by Gemini:")
    for i, c in enumerate(cities, 1):
      print(f"{i}. {c}")

else:
    pass


In [ ]:
def get_cost_matrix(api_key, locations, mode="walking", metric="distance"):
    """
    Builds the cost matrix using Google Maps.

    :param locations: List of strings (addresses or "lat,lng")
    :param mode: "driving", "walking", "bicycling", "transit"
    :param metric: "duration" (seconds) or "distance" (meters)
    :return: dist[n][n] matrix
    """
    gmaps = googlemaps.Client(key=api_key)

    response = gmaps.distance_matrix(
        origins=locations,
        destinations=locations,
        mode=mode,
        units="metric"
    )

    n = len(locations)
    dist_matrix = [[0.0] * n for _ in range(n)]

    for i, row in enumerate(response["rows"]):
        for j, element in enumerate(row["elements"]):
            if element["status"] == "OK":
                dist_matrix[i][j] = element[metric]["value"]
            else:
                dist_matrix[i][j] = float("inf")

    return dist_matrix


dist_matrix = get_cost_matrix(
    MAPS_API_KEY,
    cities,
    mode=travel_mode,
    metric=cost_metric
)

print("Cost matrix:")
dist_matrix


In [73]:
def held_karp1(dist):
    n = len(dist)
    N = 1 << n
    INF = float("inf")

    dp = [[INF] * n for _ in range(N)]
    parent = [[None] * n for _ in range(N)]

    dp[1][0] = 0

    for mask in range(1, N):
        if not (mask & 1):
            continue
        for j in range(1, n):
            if not (mask & (1 << j)):
                continue
            prev_mask = mask ^ (1 << j)
            for k in range(n):
                if prev_mask & (1 << k):
                    cost = dp[prev_mask][k] + dist[k][j]
                    if cost < dp[mask][j]:
                        dp[mask][j] = cost
                        parent[mask][j] = k

    full_mask = (1 << n) - 1
    min_cost = INF
    last = None

    for j in range(1, n):
        cost = dp[full_mask][j] + dist[j][0]
        if cost < min_cost:
            min_cost = cost
            last = j

    path = []
    mask = full_mask
    curr = last

    while curr is not None:
        path.append(curr)
        prev = parent[mask][curr]
        mask ^= (1 << curr)
        curr = prev

    path.reverse()
    path.append(0)

    return min_cost, path


def held_karp2(dist):
    n = len(dist)
    N = 1 << n
    INF = float("inf")

    dp = [[INF] * n for _ in range(N)]
    parent = [[None] * n for _ in range(N)]

    dp[1][0] = 0

    for mask in range(1, N):
        if not (mask & 1):
            continue
        for j in range(1, n):
            if not (mask & (1 << j)):
                continue
            prev_mask = mask ^ (1 << j)
            for k in range(n):
                if prev_mask & (1 << k):
                    cost = dp[prev_mask][k] + dist[k][j]
                    if cost < dp[mask][j]:
                        dp[mask][j] = cost
                        parent[mask][j] = k

    full_mask = (1 << n) - 1
    min_cost = INF
    last = None

    for j in range(1, n):
        cost = dp[full_mask][j]
        if cost < min_cost:
            min_cost = cost
            last = j

    path = []
    mask = full_mask
    curr = last

    while curr is not None:
        path.append(curr)
        prev = parent[mask][curr]
        mask ^= (1 << curr)
        curr = prev

    path.reverse()

    return min_cost, path


In [ ]:
import urllib.parse

# TSP algorithm mapping for dynamic selection
tsp_functions = {1: held_karp1, 2: held_karp2}
selected_algorithm = tsp_functions[PROBLEM_TYPE]
min_cost, path = selected_algorithm(dist_matrix)

# Result formatting
if cost_metric == "distance":
    formatted_value = min_cost / 1000
    unit = "km"
else:
    formatted_value = min_cost / 60
    unit = "min"

print(f"--- OPTIMIZATION RESULT (Mode: {'Cycle' if PROBLEM_TYPE == 1 else 'Path'}) ---")
print(f"Total travel cost: {formatted_value:.2f} {unit}")
print("Stop sequence:")
for i, stop_idx in enumerate(path, 1):
    print(f"{i}. {cities[stop_idx]}")


# --- GOOGLE MAPS URL BUILDING ---

base_url = "https://www.google.com/maps/dir/?api=1"
origin = cities[path[0]]
destination = cities[path[-1]]

# Waypoints: all intermediate cities
if len(path) > 2:
    intermediate_stops = [cities[i] for i in path[1:-1]]
    waypoints = "|".join(intermediate_stops)
else:
    waypoints = ""

params = {
    "origin": origin,
    "destination": destination,
    "waypoints": waypoints,
    "travelmode": travel_mode
}

google_maps_link = f"{base_url}&{urllib.parse.urlencode(params)}"

print("\nMAPS LINK:")
print(google_maps_link)